In [1]:
import json
from utils.utils import *
from langchain_ollama import ChatOllama
from pydantic import BaseModel

In [2]:
with open("sample_notebooks/iris_classification.ipynb", "r") as f:
    notebook = json.load(f)

In [3]:
with open("temp/code_from_notebook.txt", "w") as f:
    for cell in notebook['cells']:
        for line in cell['source']:
            f.write(line)
        f.write("\n")

In [21]:
# llm_name = "llama3.2:1b"
llm_name = "codellama"
embedding_model = "nomic-embed-text"

In [5]:
data = ingest_text_file("temp/code_from_notebook.txt")

Text file loaded.


In [6]:
print(data)

import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

import matplotlib.pyplot as plt
data = load_iris()
X = data['data']
y = data['target']
labels = data['target_names']
feature_names = data['feature_names']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7)
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_true=y_test, y_pred=y_pred, target_names=labels))
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
ConfusionMatrixDisplay(cm, display_labels=labels).plot()
plt.show()



In [7]:
# Split the documents into chunks
chunks = split_documents(data, chunk_size=1000, chunk_overlap=300)

In [8]:
# Create the vector database
vector_db = create_vector_db(chunks, embedding_model)

Vector database created.


In [22]:
# Initialize the language model
class notebook_check_result(BaseModel):
    code_for_confusion_matrix: bool
    code_for_classification_matrix: bool

llm = ChatOllama(model=llm_name, temperature=0, format=notebook_check_result.model_json_schema())

In [23]:
# Create the retriever
retriever = create_retriever(vector_db, llm)

Retriever created.


In [24]:
# Create the chain with preserved syntax
chain = create_chain(retriever, llm)

Chain created successfully.


In [26]:
# Example query
question = "Does the given code contain code for confusion matrix and classification report?"

# Get the response
res = chain.invoke(input=question)
print("Response:")
print(res)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Response:
{
"code_for_classification_matrix": true,
"code_for_confusion_matrix": true
}
